<a href="https://colab.research.google.com/github/shivams289/Projects/blob/main/Indian_bonds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Here we will be comparing after tax return on bonds listed on NSE


1.   We reomve unwanted columns
2.   we clean the data from extra things
1.   We change the datatype of numerical columms to float
2.   We use datetime format for maturity dates, so that we can use this in our calculation
1.  Present value of bonds cash flows`(PV0)`: `PV0 = (C*{1 - (1+r)**(-n) })/r + C*FV/(1+r)^(-n)`, where `C` is coupon rate and `n` is time to maturty, `r` is the yield to maturity(YTM), and `FV` is the bonds face value.
1.   We will use numpy financial library for IRR calculation of our cash flow
1.   Calculation is not perfect as our YTM column which we calculate in this is an approximation














In [1]:
pip install numpy-financial

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import math
from datetime import datetime
import tqdm as tqdm
import numpy_financial as npf

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
df = pd.read_csv('/content/drive/MyDrive/Data/bonds.csv')

In [5]:
df.columns

Index(['SYMBOL \n', 'SERIES \n', 'BOND TYPE \n', 'COUPON RATE \n',
       'FACE VALUE \n', 'LTP \n', '%CHNG \n', 'VOLUME \n(Shares)', 'VALUE ',
       ' \n', 'CREDIT RATING \n', 'MATURITY DATE \n'],
      dtype='object')

In [6]:
df.drop(['%CHNG \n', ' \n' , 'VALUE ' ], axis = 1, inplace =True)
df.columns

Index(['SYMBOL \n', 'SERIES \n', 'BOND TYPE \n', 'COUPON RATE \n',
       'FACE VALUE \n', 'LTP \n', 'VOLUME \n(Shares)', 'CREDIT RATING \n',
       'MATURITY DATE \n'],
      dtype='object')

In [7]:
df.columns = ['symbol', 'series', 'bond_type', 'coupon', 'face_value', 'ltp', 'volume', 'credit_rating', 'maturity_date' ]

In [8]:
#Dropping commas so that we can change the datatype to float
df.replace(',','', regex=True, inplace=True)

In [9]:
#Dropping '-' so that we can convert in datetime format
df.drop(df[df['maturity_date'] == '-'].index, inplace = True)

In [10]:
df.drop(df[df['coupon'] == '-'].index, inplace = True)

In [11]:
df.drop(df[df['ltp']=='-'].index, inplace = True)

In [12]:
df[['face_value', 'volume', 'ltp', 'coupon']] = df[['face_value', 'volume', 'ltp', 'coupon']].astype(float)

In [13]:
df.drop(df[df['volume'] == 0].index, inplace =True)

In [14]:
df['maturity_date'] = pd.to_datetime(df['maturity_date'], infer_datetime_format = True)

In [15]:
set(df.bond_type)

{'Regular', 'Tax Free', 'Taxable'}

In [16]:
#We will assign Bond type num =0 for Tax free bonds so that it will be helpful when we aplly 
#taxes to calculate our real return
df['bond_type_num'] = df['bond_type'].apply(lambda x: 0 if x == "Tax Free" else 1)

In [17]:
df['Interest_perc'] = (df['coupon']*df['face_value'])/df['ltp']

In [18]:
df['After_tax_interest_perc'] =  df['Interest_perc']*(1 - df['bond_type_num']*0.3) 

In [19]:
df[(df['After_tax_interest_perc']>=6) & (df['volume'] >= 100)]

,symbol,series,bond_type,coupon,face_value,ltp,volume,credit_rating,maturity_date,bond_type_num,Interest_perc,After_tax_interest_perc
0,IIHFL,N5,Regular,9.60,1000.0,988.00,5609.0,CRISIL AA Stable / BWR AA+ Negative,2028-11-03,1,9.716599,6.801619
2,SRTRANSFIN,YL,Regular,9.40,1000.0,1065.00,2177.0,CRISIL AA+ STABLE / IND AA+ STABLE,2028-07-12,1,8.826291,6.178404
3,ECLFINANCE,NK,Regular,9.85,1000.0,894.75,2580.0,CRISIL AA STABLE / ICRA AA STABLE,2028-08-06,1,11.008662,7.706063
4,IFCI,NL,Regular,9.40,1000.0,1056.00,1612.0,ICRA A STABLE / BWR AA- STABLE,2025-02-13,1,8.901515,6.231061
6,IIFL,NF,Regular,9.60,1000.0,993.00,1673.0,CRISIL AA Negative / BWR AA+ Negative,2028-06-24,1,9.667674,6.767372
7,IIHFL,N4,Regular,10.00,1000.0,992.10,1340.0,CRISIL AA Stable / BWR AA+ Negative,2028-11-03,1,10.079629,7.055740
12,PFC,N8,Tax Free,8.92,1000.0,1436.00,754.0,AAA,2033-11-16,0,6.211699,6.211699
14,IIFL,N7,Regular,10.50,1000.0,1045.00,895.0,CRISIL AA STABLE / ICRA AA STABLE / BWR AA+ ST...,2029-02-07,1,10.047847,7.033493
16,IIFCL,N4,Tax Free,8.91,1000.0,1419.00,582.0,AAA/Stable,2034-01-22,0,6.279070,6.279070
18,HUDCO,N9,Tax Free,7.69,1000.0,1269.99,530.0,AAA,2031-03-15,0,6.055166,6.055166


In [20]:
# n>=2 condition is applied coz program gave arror when n was less than 1 or 1 as we are running the loop in range(n-1)
def compute_irr(n, fv, ltp, coupon):
  if n>=2:
    lst = [coupon for i in range(n-1)]
    lst.append(fv + lst[0])
    lst.insert(0, -ltp)
    irr = npf.irr(lst)
  else:
    irr = npf.irr([-ltp, (fv+coupon)])
    
  return irr*100

In [21]:
print(compute_irr(6, 100.1, 92.2, 9.1))

10.953944017231155


In [22]:
# Index row 50 was missing so I was getting key error 50 when applying lambda fundtion below
df.reset_index(inplace = True)

In [23]:
lst = []
for i in range(len(df)):
  x = int(df['maturity_date'][i].year)
  lst.append(x)
  

In [24]:
dfm = pd.DataFrame(lst, columns= ['Maturity_year'])

In [25]:
df = pd.concat([df, dfm], axis = 'columns', sort = False)

In [26]:
df['TTM'] = df['Maturity_year'] - int(datetime.today().year)


In [27]:
df.drop(df[df['TTM']==0].index, inplace = True)

In [28]:
# This Yiled is not perfect as we are taking TTM(time to maturity): starting of 2021 to end of maturity year
df['Yield'] = df.apply(lambda x: compute_irr(x['TTM'], x['face_value'], x['ltp'], (x['coupon']*x['face_value'])/100), axis = 1)

In [30]:
df['After_tax_yield_perc'] = df['Yield']*(1 - df['bond_type_num']*0.3) 

In [37]:
df[(df['After_tax_yield_perc']>=100)  ]

,index,symbol,series,bond_type,coupon,face_value,ltp,volume,credit_rating,maturity_date,bond_type_num,Interest_perc,After_tax_interest_perc,Maturity_year,TTM,Yield,After_tax_yield_perc
100,109,SREIBNPNCD,Y2,Regular,9.50,1000.0,320.00,75.0,BWR AA+ Stable,2022-02-27,1,29.687500,20.781250,2022,1,242.187500,169.531250
108,119,SREIBNPNCD,NL,Regular,9.75,1000.0,345.00,33.0,BWR AA+ STABLE / ACUITE AA STABLE,2022-01-17,1,28.260870,19.782609,2022,1,218.115942,152.681159
110,121,SREIBNPNCD,NJ,Regular,9.35,1000.0,310.00,31.0,BWR AA+ STABLE / ACUITE AA STABLE,2022-01-17,1,30.161290,21.112903,2022,1,252.741935,176.919355
114,127,SREIBNPNCD,Y1,Regular,9.12,1000.0,349.98,7.0,BWR AA+ Stable,2022-02-27,1,26.058632,18.241042,2022,1,211.789245,148.252472
115,129,RHFL,N4,Regular,9.05,1000.0,228.00,6.0,CARE AA+ STABLE / BWR AA+ STABLE,2022-01-03,1,39.692982,27.785088,2022,1,378.289474,264.802632
